In [75]:
%%capture
%pip install yfinance

In [76]:
import yfinance as yf
import datetime

In [77]:
df = yf.download(['^NSEI'], period='max')
df.head()

C:\Users\ishui\AppData\Local\Temp\ipykernel_27732\2959623623.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(['^NSEI'], period='max')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^NSEI,^NSEI,^NSEI,^NSEI,^NSEI
Date,,,,,
2007-09-17,4494.649902,4549.049805,4482.850098,4518.450195,0
2007-09-18,4546.200195,4551.799805,4481.549805,4494.100098,0
2007-09-19,4732.350098,4739.000000,4550.250000,4550.250000,0
2007-09-20,4747.549805,4760.850098,4721.149902,4734.850098,0
2007-09-21,4837.549805,4855.700195,4733.700195,4752.950195,0


In [78]:
closing_prices = df['Close']
print(closing_prices.info())
closing_prices.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4386 entries, 2007-09-17 to 2025-08-04
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ^NSEI   4386 non-null   float64
dtypes: float64(1)
memory usage: 68.5 KB
None


Ticker,^NSEI
Date,
2007-09-17,4494.649902
2007-09-18,4546.200195
2007-09-19,4732.350098
2007-09-20,4747.549805
2007-09-21,4837.549805


In [79]:
closing_prices.dropna(inplace=True)
print(len(closing_prices))

4386


C:\Users\ishui\AppData\Local\Temp\ipykernel_27732\646364951.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closing_prices.dropna(inplace=True)


In [80]:
closing_prices["nifty_pct_change"] = closing_prices["^NSEI"].pct_change() * 100

C:\Users\ishui\AppData\Local\Temp\ipykernel_27732\328362761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closing_prices["nifty_pct_change"] = closing_prices["^NSEI"].pct_change() * 100


In [81]:
closing_prices

Ticker,^NSEI,nifty_pct_change
Date,,
2007-09-17,4494.649902,NaN
2007-09-18,4546.200195,1.146926
2007-09-19,4732.350098,4.094626
2007-09-20,4747.549805,0.321187
2007-09-21,4837.549805,1.895715
...,...,...
2025-07-29,24821.099609,0.568047
2025-07-30,24855.050781,0.136784
2025-07-31,24768.349609,-0.348827


In [82]:
def calculate_returns_SIP(closing_prices, start_date, budget, investment_chunk):
    print(f"Calculating SIP returns from {start_date}, with investment chunk = {investment_chunk} and budget = {budget}")
    df = closing_prices[start_date:]
    corpus = 0
    deposit_count = 0
    prev_month = 0
    invested_amt = 0
    for row in df.iterrows():
        curr_month = row[0].date().month
        if invested_amt < budget and prev_month != curr_month:
            corpus += investment_chunk
            invested_amt += investment_chunk
            deposit_count += 1
            prev_month = curr_month
        corpus *= (100 + row[1]['nifty_pct_change'])/100

    returns = corpus - invested_amt
    returns_pct = returns/invested_amt * 100
    print("Number of installments:", deposit_count)
    print("Invested corpus:", invested_amt)
    print("Total corpus at end:", corpus)
    print("Returns:", returns)
    print("Returns %:", returns_pct, "%")
    return returns, returns_pct

In [83]:
def calculate_returns_lumpsum(closing_prices, start_date, budget):
    print(f"Calculating lumpsum returns from {start_date} with budget = {budget}")
    df = closing_prices[start_date:]
    corpus = budget
    for row in df.iterrows():
        corpus *= (100 + row[1]['nifty_pct_change'])/100
    returns = corpus - budget
    returns_pct = returns/budget * 100
    print("Invested corpus:", budget)
    print("Total corpus at end:", corpus)
    print("Returns:", returns)
    print("Returns %:", returns_pct, "%")
    return returns, returns_pct

In [84]:
# Invest chunk whenever price drops by 1% in a day
def calculate_returns_dip(closing_prices, start_date, budget, investment_chunk):
    print(f"Calculating dip buying returns from {start_date}, with investment chunk = {investment_chunk} and budget = {budget}")
    print("Investing every time index falls >1%")
    df = closing_prices[start_date:]
    corpus = 0
    deposit_count = 0
    invested_amt = 0
    for row in df.iterrows():
        corpus *= (100 + row[1]['nifty_pct_change'])/100
        if invested_amt < budget and row[1]['nifty_pct_change'] < -1.0:
            corpus += investment_chunk
            invested_amt += investment_chunk
            deposit_count += 1
    returns = corpus - invested_amt
    returns_pct = returns/invested_amt * 100
    print("Number of installments:", deposit_count)
    print("Invested corpus:", invested_amt)
    print("Total corpus at end:", corpus)
    print("Returns:", returns)
    print("Returns %:", returns_pct, "%")
    return returns, returns_pct


In [90]:
BUDGET = 100000 # 1 lakhs
INVESTMENT_CHUNK = 10000
NUM_SIMULATIONS = 250
RANGE_START = datetime.date(2010, 1, 1)
RANGE_END = datetime.date.today()

In [91]:
import random
def get_random_dates(start, end, num_dates):
    dates_bet = end - start
    total_days = dates_bet.days

    res = []
    for idx in range(num_dates):
        random.seed(a=None)
        
        # getting random days
        randay = random.randrange(total_days)
        
        # getting random dates 
        res.append(start + datetime.timedelta(days=randay))

    return res

In [92]:
start_dates = get_random_dates(RANGE_START, RANGE_END, NUM_SIMULATIONS)
win_counts = {
        "lumpsum": 0,
        "sip": 0,
        "dip": 0
    }
for date in start_dates:
    print("\n-----------------------------------------")
    print("START DATE:", date)
    lumpsum_returns, _ = calculate_returns_lumpsum(closing_prices, date, BUDGET)
    sip_returns, _ = calculate_returns_SIP(closing_prices, date, BUDGET, INVESTMENT_CHUNK)
    dip_returns, _ = calculate_returns_dip(closing_prices, date, BUDGET, INVESTMENT_CHUNK)

    if lumpsum_returns > sip_returns and lumpsum_returns > dip_returns:
        print("Lumpsum investing wins!")
        win_counts["lumpsum"] += 1
    elif sip_returns > lumpsum_returns and sip_returns > dip_returns:
        print("SIP investing wins!")
        win_counts["sip"] += 1
    else:
        print("dip investing wins!")
        win_counts["dip"] += 1

print(win_counts)


-----------------------------------------
START DATE: 2017-12-02
Calculating lumpsum returns from 2017-12-02 with budget = 100000
Invested corpus: 100000
Total corpus at end: 244252.5092083993
Returns: 144252.5092083993
Returns %: 144.25250920839932 %
Calculating SIP returns from 2017-12-02, with investment chunk = 10000 and budget = 100000
Number of installments: 10
Invested corpus: 100000
Total corpus at end: 230385.5387997661
Returns: 130385.5387997661
Returns %: 130.3855387997661 %
Calculating dip buying returns from 2017-12-02, with investment chunk = 10000 and budget = 100000
Investing every time index falls >1%
Number of installments: 10
Invested corpus: 100000
Total corpus at end: 234899.39247502887
Returns: 134899.39247502887
Returns %: 134.89939247502886 %
Lumpsum investing wins!

-----------------------------------------
START DATE: 2020-07-18
Calculating lumpsum returns from 2020-07-18 with budget = 100000
Invested corpus: 100000
Total corpus at end: 226778.84694196834
Ret

C:\Users\ishui\AppData\Local\Temp\ipykernel_27732\1797714830.py:16: RuntimeWarning: invalid value encountered in scalar divide
  returns_pct = returns/invested_amt * 100


Number of installments: 10
Invested corpus: 100000
Total corpus at end: 444806.0001423672
Returns: 344806.0001423672
Returns %: 344.8060001423672 %
Calculating dip buying returns from 2010-04-15, with investment chunk = 10000 and budget = 100000
Investing every time index falls >1%
Number of installments: 10
Invested corpus: 100000
Total corpus at end: 487637.99482964084
Returns: 387637.99482964084
Returns %: 387.63799482964083 %
dip investing wins!

-----------------------------------------
START DATE: 2024-05-15
Calculating lumpsum returns from 2024-05-15 with budget = 100000
Invested corpus: 100000
Total corpus at end: 111274.2701686487
Returns: 11274.270168648698
Returns %: 11.274270168648698 %
Calculating SIP returns from 2024-05-15, with investment chunk = 10000 and budget = 100000
Number of installments: 10
Invested corpus: 100000
Total corpus at end: 103111.88431316639
Returns: 3111.884313166389
Returns %: 3.1118843131663887 %
Calculating dip buying returns from 2024-05-15, wit

# Summary

In [95]:
print(f"Total simulations: {NUM_SIMULATIONS}")
print(f"Lumpsum investing won {win_counts["lumpsum"]} times: {(win_counts["lumpsum"]/NUM_SIMULATIONS) * 100}%")
print(f"SIP investing won {win_counts["sip"]} times: {(win_counts["sip"]/NUM_SIMULATIONS) * 100}%")
print(f"Dip investing won {win_counts["dip"]} times: {(win_counts["dip"]/NUM_SIMULATIONS) * 100}%")

Total simulations: 250
Lumpsum investing won 143 times: 57.199999999999996%
SIP investing won 35 times: 14.000000000000002%
Dip investing won 72 times: 28.799999999999997%
